This is the main notebook of the report, it'll contain all of our findings and developments.

# Word2VecAverage

In [1]:
import DataReader
import Word2VecAverage
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

x_test_set, x_train_set, y_test_set, y_train_set = DataReader.split_sheet_into_test_training_per_word()
w2v = Word2VecAverage.Word2VecAverage()
w2v.train_word2vec()

x_test_features = w2v.create_feature_set(x_test_set)
x_train_features = w2v.create_feature_set(x_train_set)

clf = LogisticRegression()
clf.fit(x_test_features, y_test_set)
test_pred_clf = clf.predict(x_test_features)
print(metrics.accuracy_score(test_pred_clf, y_test_set))

C:\Users\lolco\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\lolco\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6827116545426405


# Doc2VecBasic

In [2]:
import Doc2VecFeatureCreator
import DataReader
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

d2v = Doc2VecFeatureCreator.Doc2VecFeatureCreator()
x_test_set, x_train_set, y_test_set, y_train_set = DataReader.split_sheet_into_test_training_per_word()

x_train_features = d2v.create_feature_set(x_train_set)
x_test_features = d2v.create_feature_set(x_test_set)

In [3]:
clf = LogisticRegression()
clf.fit(x_test_features, y_test_set)
test_pred_clf = clf.predict(x_test_features)
print(metrics.accuracy_score(test_pred_clf, y_test_set))

C:\Users\lolco\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.6745085900015477


# Ngram

In [4]:
import Ngram
from DataReader import split_sheet_into_test_training_per_word
x_test, x_train, y_test, y_train = split_sheet_into_test_training_per_word()
n=2

One model:

In [5]:
x_clean = Ngram.process_1(x_train, y_train, n)
model = Ngram.get_model( x_clean,n)
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, model, n))

correct, error, doesn't know (in%)


ValueError: 'rifle' is not in list

Multiple models (one per definition):

In [ ]:
x1,x2,x3,x4 = Ngram.process_4(x_train, y_train, n)
models = []
xs = [x1,x2,x3,x4]
for x in xs:
    models.append(Ngram.get_model(x, n))
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, models, n))

# BoW approach

In [ ]:
import Bow
mod = Bow.Bow(x_train, y_train)
print("correct, wrong, noidea")
print(mod.get_score(x_test,y_test))

# Baseline

In [ ]:
from MostCommon import MostCommon
from sklearn import metrics
import numpy as np
mc = MostCommon()
counts = mc.createCounts()
predicted = mc.classify(mc.x_test, counts)
testl = np.ravel(mc.y_test)
print(metrics.classification_report(testl, predicted))

# Semantic dependencies

In [ ]:
from SemanticDep import SemanticDep
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
sd = SemanticDep()
f = sd.createFeatures(sd.x_train)
l = np.ravel(sd.y_train)
testf = sd.createFeatures(sd.x_test)
testl = np.ravel(sd.y_test)

# train the model on the features
clf = MultinomialNB()
clf.fit(f, l)

# predict the labels for the test data
predicted = clf.predict(testf)
# output your results
print(metrics.classification_report(testl, predicted))